target: ¿Pagó la cuota? | Sí | Pendiente (entra en Mora)

In [144]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Configuracion

In [145]:
import pandas as pd
import unidecode

# Set Pandas to display all columns in the DataFrame
pd.set_option('display.max_columns', None)

# Funciones

In [146]:
# Función para limpiar nombres de columnas
def clean_column_names(columns):
    return [unidecode.unidecode(col.upper().replace(" ", "_")) for col in columns]

In [147]:
# Función para mostrar las frecuencias absolutas y relativas de un campo
def frecuencia_categoria(df,columna,orden):
    resultado = df.groupby([columna]).agg(CANTIDAD=(columna, 'count')).reset_index()
    total = resultado['CANTIDAD'].sum()
    resultado['% TOTAL'] = (resultado['CANTIDAD'] / total) * 100
    if orden == 0:
        resultado = resultado.sort_values('% TOTAL', ascending=False)
        return resultado
    elif orden == 1:
        resultado = resultado.sort_values(columna, ascending=True)
        return resultado
    else:
        return print('En orden solo puede escoger los valores 0,1')
        


# Función para graficar los campos categóricos
# Recomendaciones ncol=2, color='rocket',grafico=barplot
def plot_categoria(df,lista_campos,ncol,color,ancho,largo):
    # Calcular el número de filas y columnas para organizar los subgráficos
    num_filas = (len(lista_campos) + 1) // ncol
    num_columnas = ncol if len(lista_campos) > 1 else 1

    # Crear la figura y los subgráficos
    fig, axs = plt.subplots(num_filas, num_columnas, figsize=(ancho, largo))

    # Ajustar los espacios entre los subgráficos
    plt.subplots_adjust(hspace=0.2)

    # Iterar sobre los encabezados y crear un gráfico para cada columna del DataFrame
    for i, encabezado in enumerate(lista_campos):
        fila = i // ncol
        columna = i % ncol if num_columnas > 1 else 0
        ax = axs[fila, columna] if num_filas > 1 else axs[columna]
        counts = df[encabezado].value_counts()  # Obtener conteo de valores
        sns.barplot(x=counts.values, y=counts.index, palette=color, ax=ax)
        ax.set_title(f"CATEGORÍA {encabezado}")
        ax.set_ylabel("")  # Eliminar título del eje y

    return plt.show()

def var_nan_flag(df, columna, null="None"):
    new_col_name = columna + '_flag'
    if null=="None":

        df = (
            df
            .with_columns(
                pl.when(pl.col(columna).is_null())
                .then(pl.lit(1))
                .otherwise(pl.lit(0))
                .alias(new_col_name)
            )
        )
    
    elif null!="None":
        df = (
            df
            .with_columns(
                pl.when(pl.col(columna)==null)
                .then(pl.lit(1))
                .otherwise(pl.lit(0))
                .alias(new_col_name)
            )
        )

    return df

def missing_values_table(df):

    mis_val = df.select(pl.all().is_null().sum())
    mis_val_percent = 100 * mis_val / len(df)

    mis_val = mis_val.row(0)
    mis_val_percent = mis_val_percent.row(0)
    mis_val_columnas = df.columns

    mis_val_table = (

        pl.DataFrame(
            {
                'columnas':mis_val_columnas,
                'num_missings':mis_val,
                '% missings':mis_val_percent
            }
        )
        .filter(pl.col('num_missings')!=0)
        .sort('num_missings', descending=True)
    )
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
        "There are " + str(mis_val_table.shape[0]) +
          " columns that have missing values.")

    return mis_val_table, mis_val_table.get_column('columnas').to_list()

def reemplazar_por_mediana(x,limite_superior,limite_inferior,mediana):
    if (x>limite_superior)|(x<limite_inferior):
        return mediana
    else:
        return x
    
def porcentaje_outlier(df, column):
    rango_intercuartil = df.get_column(column).quantile(0.75) - df.get_column(column).quantile(0.25)
    limite_superior = df.get_column(column).quantile(0.75) + 1.5*rango_intercuartil
    limite_inferior = df.get_column(column).quantile(0.25) - 1.5*rango_intercuartil
    

    outliers_count = df.shape[0] - df.filter(pl.col(column).is_between(limite_inferior,limite_superior)).shape[0]

    return limite_superior,limite_inferior, np.round(outliers_count*100/df.shape[0],2)
    
def tratar_outlier(df, column, min=0.0001,max=0.95):

    limite_superior,limite_inferior, porcentaje = porcentaje_outlier(df, column)
        
    if (porcentaje>=5):

        percentil_95 = df.get_column(column).quantile(max)
        percentil_2 = df.get_column(column).quantile(min)
        df = df.with_columns(pl.col(column).map_elements(lambda x: percentil_95 if x > percentil_95 else x))
        df = df.with_columns(pl.col(column).map_elements(lambda x: percentil_2 if x < percentil_2 else x))
        return df
    else:
        return df
    
def reemplazar_negativo(df, columna):

    return (
        df
        .with_columns(pl.when(pl.col(columna)<0).then(pl.lit(0)).otherwise(pl.col(columna)).alias(columna))
    )

def preprocess_data(df, target_column):
    # Identify categorical columns
    cat_columns = df.select_dtypes(include=['object', 'category']).columns
    cat_columns = [col for col in cat_columns if col != target_column]
    
    # Handle missing values
    for col in cat_columns:
        df[col].fillna('Unknown', inplace=True)
    
    print("Preprocessing completed.")
    return df, cat_columns


def analyze_categorical(df, target_column, cat_columns):
    results = {}
    for col in cat_columns:
        # Frequency distribution
        freq = df[col].value_counts(normalize=True)
        
        # Target rate analysis
        target_rates = df.groupby(col)[target_column].mean()
        # Chi-square test
        contingency_table = pd.crosstab(df[col], df[target_column])
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
                
        # Information Value
        iv = calculate_iv(df, col, target_column)
        
        results[col] = {
            'frequency': freq,
            'target_rates': target_rates,
            'chi2_p_value': p_value,
            'iv': iv
        }
    
    return results


def calculate_iv(df, feature, target):
    eps = 1e-10  # Small value to prevent division by zero
    grouped = df.groupby(feature)[target].agg(['count', 'sum'])
    grouped['non_event'] = grouped['count'] - grouped['sum']
    grouped['f_perc'] = grouped['count'] / grouped['count'].sum()
    grouped['non_event_perc'] = (grouped['non_event'] + eps) / (grouped['non_event'].sum() + eps)
    grouped['event_perc'] = (grouped['sum'] + eps) / (grouped['sum'].sum() + eps)
    grouped['woe'] = np.log(grouped['event_perc'] / grouped['non_event_perc'])
    grouped['iv'] = (grouped['event_perc'] - grouped['non_event_perc']) * grouped['woe']
    iv = grouped['iv'].sum()
    return iv if np.isfinite(iv) else None

def visualize_results(df, analysis_results, target_column):
    for col, result in analysis_results.items():
        plt.figure(figsize=(16, 6))
        
        # Frequency plot
        plt.subplot(131)
        result['frequency'].plot(kind='bar')
        plt.title(f'Frequency Distribution - {col}')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45, ha='right')
        
        # Target rate plot
        plt.subplot(132)
        result['target_rates'].plot(kind='bar')
        plt.title(f'Target Rate - {col}')
        plt.ylabel('Target Rate')
        plt.xticks(rotation=45, ha='right')
        
        # Mosaic plot
        plt.subplot(133)
        contingency_table = pd.crosstab(df[col], df[target_column])
        sns.heatmap(contingency_table, annot=True, fmt='d', cmap='YlGnBu')
        plt.title(f'Mosaic Plot - {col} vs Target')
        plt.ylabel(col)
        plt.xlabel('Target')
        
        plt.tight_layout()
        plt.show()

def feature_importance(df, target_column, cat_columns):
    le = LabelEncoder()
    X = df[cat_columns].apply(le.fit_transform)
    y = df[target_column]

    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X, y)

    importances = pd.Series(rf.feature_importances_, index=cat_columns).sort_values(ascending=False)
    plt.figure(figsize=(10, 6))
    importances.plot(kind='bar')
    plt.title('Feature Importance')
    plt.ylabel('Importance')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    return importances

def mutual_information(df, target_column, cat_columns):
    le = LabelEncoder()
    X = df[cat_columns].apply(le.fit_transform)
    y = df[target_column]

    mi_scores = mutual_info_classif(X, y)
    mi_scores = pd.Series(mi_scores, index=cat_columns).sort_values(ascending=False)
    plt.figure(figsize=(10, 6))
    mi_scores.plot(kind='bar')
    plt.title('Mutual Information Scores')
    plt.ylabel('Mutual Information')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    return mi_scores

def correlation_analysis(df, cat_columns):
    # One-hot encode categorical variables
    #onehot = OneHotEncoder()
    #encoded = onehot.fit_transform(df[cat_columns])
    #encoded_df = pd.DataFrame(encoded, columns=onehot.get_feature_names_out(cat_columns))
    encoded_df = pd.get_dummies(df, cat_columns)
    
    # Calculate correlation matrix
    corr_matrix = encoded_df.corr()
    
    # Plot heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, cmap='coolwarm', annot=False, fmt='.2f', linewidths=0.5)
    plt.title('Correlation Heatmap of Encoded Categorical Variables')
    plt.tight_layout()
    plt.show()

def target_encoding_performance(df, target_column, cat_columns):
    # Perform target encoding
    encoded_df = df.copy()
    for col in cat_columns:
        target_mean = df.groupby(col)[target_column].mean()
        encoded_df[f'{col}_encoded'] = df[col].map(target_mean)
    
    # Prepare data for modeling
    X = encoded_df[[f'{col}_encoded' for col in cat_columns]]
    y = encoded_df[target_column]
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train logistic regression
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate performance
    train_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
    test_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    print(f"Logistic Regression performance after target encoding:")
    print(f"Train AUC: {train_auc:.4f}")
    print(f"Test AUC: {test_auc:.4f}")
    
    # Feature importance based on logistic regression coefficients
    feature_importance = pd.Series(np.abs(model.coef_[0]), index=X.columns).sort_values(ascending=False)
    plt.figure(figsize=(10, 6))
    feature_importance.plot(kind='bar')
    plt.title('Feature Importance after Target Encoding')
    plt.ylabel('Absolute Coefficient Value')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

def mca_analysis(df, cat_columns):
    # Perform MCA
    mca = MCA(n_components=2, random_state=42)
    mca_coords = mca.fit_transform(df[cat_columns])
    
    # Plot results
    plt.figure(figsize=(10, 8))
    plt.scatter(mca_coords.iloc[:, 0], mca_coords.iloc[:, 1], alpha=0.7)
    plt.title('Multiple Correspondence Analysis')
    plt.xlabel('First Component')
    plt.ylabel('Second Component')
    plt.tight_layout()
    plt.show()

def analyze_interactions(df, target_column, cat_columns):
    for i, col1 in enumerate(cat_columns):
        for col2 in cat_columns[i+1:]:
            plt.figure(figsize=(12, 8))
            interaction_data = df.groupby([col1, col2])[target_column].mean().unstack()
            sns.heatmap(interaction_data, annot=True, fmt='.2f', cmap='YlGnBu')
            plt.title(f'Interaction Effect: {col1} vs {col2}')
            plt.tight_layout()
            plt.show()
            
def create_summary_table(analysis_results, importances, mi_scores):
    summary = []
    for col, result in analysis_results.items():
        summary.append({
            'Variable': col,
            'Chi-square p-value': result['chi2_p_value'],
            'Information Value': result['iv'],
            'Feature Importance': importances.get(col, 0),
            'Mutual Information': mi_scores.get(col, 0)
        })
    
    summary_df = pd.DataFrame(summary)
    
    # Sort by a composite score (you can adjust the weights as needed)
    summary_df['Composite Score'] = (
        summary_df['Information Value'] * 0.3 +
        summary_df['Feature Importance'] * 0.3 +
        summary_df['Mutual Information'] * 0.2 +
        (1 - summary_df['Chi-square p-value']) * 0.2
    )
    
    summary_df = summary_df.sort_values('Composite Score', ascending=False).reset_index(drop=True)
    
    return summary_df

def visualize_top_variables(summary_df, top_n=10):
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Composite Score', y='Variable', data=summary_df.head(top_n))
    plt.title(f'Top {top_n} Categorical Variables by Composite Score')
    plt.tight_layout()
    plt.show()

def function_main_plot_category(df, target_column):
    # Load and preprocess data
    df, cat_columns = preprocess_data(df, target_column)
    
    # Perform categorical variable analysis
    analysis_results = analyze_categorical(df, target_column, cat_columns)
    
    # Visualize results
    visualize_results(df, analysis_results, target_column)
    
    # Feature importance
    importances = feature_importance(df, target_column, cat_columns)
    print("Feature Importance:")
    print(importances)
    
    # Mutual Information
    mi_scores = mutual_information(df, target_column, cat_columns)
    print("\nMutual Information Scores:")
    print(mi_scores)
    
    # Correlation analysis
    correlation_analysis(df, cat_columns)
    
    # Target encoding performance
    target_encoding_performance(df, target_column, cat_columns)
    
    # MCA analysis
    mca_analysis(df, cat_columns)
    
    # Analyze interactions
    # analyze_interactions(df, target_column, cat_columns)
    summary_table = create_summary_table(analysis_results, importances, mi_scores)
    print(summary_table.head(10).to_string(index=False))
    visualize_top_variables(summary_table)
    
    

    # Print summary of findings
    print("\nSummary of Findings:")
    for col, result in analysis_results.items():
        print(f"\nVariable: {col}")
        print(f"Chi-square p-value: {result['chi2_p_value']:.4f}")
        print(f"Information Value: {result['iv']:.4f}")
        print(f"Feature Importance: {importances[col]:.4f}")
        print(f"Mutual Information: {mi_scores[col]:.4f}")
    
    print("\nRecommendations:")
    print("1. Consider keeping variables with low chi-square p-values (< 0.05).")
    print("2. Prioritize variables with higher Cramér's V, Information Value, Feature Importance, and Mutual Information scores.")
    print("3. Be cautious of variables with very high cardinality or strong correlations with other variables.")
    print("4. Review interaction effects for potential feature engineering opportunities.")
    print("5. Consult domain experts to validate these findings and consider business context.")
    
    return summary_table

# Cargando archivos

In [148]:
import pandas as pd

df_sponsor = pd.read_excel('../data/raw/BD_SPONSOR.xlsx')
df_consolidado = pd.read_excel('../data/raw/Consolidado_pagos (2).xlsx')


# df_sponsor

In [149]:
df_sponsor.head()

,Beneficiario,Nro de carpeta,DNI,Código de alumno,Categoría principal,Estado de retiros,Fecha de retiro,Fin de crédito,Fecha de fin de crédito,Periodo de egreso,Estado de contrato,Fecha de firma,Comité de riesgos,Promoción Sponsor,Universidad,Programa Académico,Carrera principal,Carrera secundaria,Dirección,Correo personal,Correo institucional,Correo laboral,Telefono personal,Telefono laboral,Telefono fijo,Fiador solidario,DNI fiador,Dirección fiador,Correo personal fiador,Telefono personal fiador,Representante legal,DNI Representante legal,Dirección representante,Correo representante,Telefono personal representante,Sexo del Alumno
0,"SALAZAR DE LA CRUZ, MARHIA DEL CARMMEN GLORIA",64,75955482.0,2220372,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-02-27,No,2024-01,USIL,CPEL,NEGOCIOS INTERNACIONALES,NINGUNA,"Psj. Arequipa 130 Urb. San Gabriel, distrito d...",msalazardelacruz9@gmail.com,marhia.salazar@usil.pe,Pendiente,51967639606,12019960,Pendiente,Gloria Estefania Vilchez Chauca,47185338,"Psj. Arequipa Mariategui Mz S12 Lt 14, distrit...",jl.gloes.9@hotmail.com,51986632214,No aplica,No aplica,No aplica,No aplica,No aplica,FEMENINO
1,"JULIAN VALENCIA, YEFERSON ORLANDO",53,70602048.0,2311639,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-02-27,Sí,2024-01,USIL,CPEL,INGENIERÍA INDUSTRIAL,NINGUNA,"Av. Los Jardines Mz N Lt 13 A.H. La Paz, distr...",yefersonorlandojulianvalencia@gmail.com,yeferson.julian@usil.pe,Pendiente,51983564306,Pendiente,Pendiente,Aldair Alexander Huarcaya Revilla,70601341,"A.H. La Paz Mz S Lt 2, distrito de Ventanilla,...",phuarcaya77@gmail.com,51929336214,No aplica,No aplica,No aplica,No aplica,No aplica,MASCULINO
2,"DE LA CRUZ SANCHEZ, MELANY LALESHCA",56,74494492.0,2220869,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-02-29,No,2024-01,USIL,CPEL,NEGOCIOS INTERNACIONALES,NINGUNA,Ca Heroes del Pueblo A.H. 10 de febrero Mz L L...,laleshckadlc@gmail.com,melany.delacruzs@usil.pe,Pendiente,51914559960,Pendiente,Pendiente,Hernan Sanchez Camargo,80365017,"Ca. Porvenir 179 Urb. Pedregal, distrito de Sa...",hernancamargo031@gmail.com,51994068382,No aplica,No aplica,No aplica,No aplica,No aplica,FEMENINO
3,"BOHORQUEZ YRAITA, VERONICA CINA",29,40187283.0,2110907,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-02-27,Sí,2024-01,USIL,CPEL,NEGOCIOS INTERNACIONALES,NINGUNA,"Psj. 5 Mz O Lt 5 Urb. Los Girasoles, distrito ...",veronicaby@hotmail.com,veronica.bohorquez@usil.pe,Pendiente,51944255319,Pendiente,Pendiente,Carlos Reinerio Tamani Rafael,46238071,"Jr. Morro Solar 1254 Dep. 401, distrito de San...",carlos.tamani@gmail.com,51984714888,No aplica,No aplica,No aplica,No aplica,No aplica,FEMENINO
4,"LOPEZ QUICAÑA, WILBER EDGAR",24,70246293.0,2321569,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-01-05,No,2024-01,USIL,CPEL,INGENIERÍA EMPRESARIAL Y DE SISTEMAS,NINGUNA,"Asoc. 8 De Noviembre Mz D Lt 04, distrito de S...",wilberedgarlopez@gmail.com,wilber.lopez@usil.pe,Pendiente,51995070696,51952564231,Pendiente,Joel Felix Sapaylle Poma,48016557,"Ca. Gerónimo De Aliaga 287 Urb. Fundo Oyague, ...",joel.sapaylle@gmail.com,51986948664,No aplica,No aplica,No aplica,No aplica,No aplica,MASCULINO


In [150]:
df_sponsor.columns = clean_column_names(df_sponsor.columns)
df_sponsor

,BENEFICIARIO,NRO_DE_CARPETA,DNI,CODIGO_DE_ALUMNO,CATEGORIA_PRINCIPAL,ESTADO_DE_RETIROS,FECHA_DE_RETIRO,FIN_DE_CREDITO,FECHA_DE_FIN_DE_CREDITO,PERIODO_DE_EGRESO,ESTADO_DE_CONTRATO,FECHA_DE_FIRMA,COMITE_DE_RIESGOS,PROMOCION_SPONSOR,UNIVERSIDAD,PROGRAMA_ACADEMICO,CARRERA_PRINCIPAL,CARRERA_SECUNDARIA,DIRECCION,CORREO_PERSONAL,CORREO_INSTITUCIONAL,CORREO_LABORAL,TELEFONO_PERSONAL,TELEFONO_LABORAL,TELEFONO_FIJO,FIADOR_SOLIDARIO,DNI_FIADOR,DIRECCION_FIADOR,CORREO_PERSONAL_FIADOR,TELEFONO_PERSONAL_FIADOR,REPRESENTANTE_LEGAL,DNI_REPRESENTANTE_LEGAL,DIRECCION_REPRESENTANTE,CORREO_REPRESENTANTE,TELEFONO_PERSONAL_REPRESENTANTE,SEXO_DEL_ALUMNO
0,"SALAZAR DE LA CRUZ, MARHIA DEL CARMMEN GLORIA",64,75955482.0,2220372,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-02-27,No,2024-01,USIL,CPEL,NEGOCIOS INTERNACIONALES,NINGUNA,"Psj. Arequipa 130 Urb. San Gabriel, distrito d...",msalazardelacruz9@gmail.com,marhia.salazar@usil.pe,Pendiente,51967639606,12019960,Pendiente,Gloria Estefania Vilchez Chauca,47185338,"Psj. Arequipa Mariategui Mz S12 Lt 14, distrit...",jl.gloes.9@hotmail.com,51986632214,No aplica,No aplica,No aplica,No aplica,No aplica,FEMENINO
1,"JULIAN VALENCIA, YEFERSON ORLANDO",53,70602048.0,2311639,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-02-27,Sí,2024-01,USIL,CPEL,INGENIERÍA INDUSTRIAL,NINGUNA,"Av. Los Jardines Mz N Lt 13 A.H. La Paz, distr...",yefersonorlandojulianvalencia@gmail.com,yeferson.julian@usil.pe,Pendiente,51983564306,Pendiente,Pendiente,Aldair Alexander Huarcaya Revilla,70601341,"A.H. La Paz Mz S Lt 2, distrito de Ventanilla,...",phuarcaya77@gmail.com,51929336214,No aplica,No aplica,No aplica,No aplica,No aplica,MASCULINO
2,"DE LA CRUZ SANCHEZ, MELANY LALESHCA",56,74494492.0,2220869,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-02-29,No,2024-01,USIL,CPEL,NEGOCIOS INTERNACIONALES,NINGUNA,Ca Heroes del Pueblo A.H. 10 de febrero Mz L L...,laleshckadlc@gmail.com,melany.delacruzs@usil.pe,Pendiente,51914559960,Pendiente,Pendiente,Hernan Sanchez Camargo,80365017,"Ca. Porvenir 179 Urb. Pedregal, distrito de Sa...",hernancamargo031@gmail.com,51994068382,No aplica,No aplica,No aplica,No aplica,No aplica,FEMENINO
3,"BOHORQUEZ YRAITA, VERONICA CINA",29,40187283.0,2110907,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-02-27,Sí,2024-01,USIL,CPEL,NEGOCIOS INTERNACIONALES,NINGUNA,"Psj. 5 Mz O Lt 5 Urb. Los Girasoles, distrito ...",veronicaby@hotmail.com,veronica.bohorquez@usil.pe,Pendiente,51944255319,Pendiente,Pendiente,Carlos Reinerio Tamani Rafael,46238071,"Jr. Morro Solar 1254 Dep. 401, distrito de San...",carlos.tamani@gmail.com,51984714888,No aplica,No aplica,No aplica,No aplica,No aplica,FEMENINO
4,"LOPEZ QUICAÑA, WILBER EDGAR",24,70246293.0,2321569,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,Firmado,2024-01-05,No,2024-01,USIL,CPEL,INGENIERÍA EMPRESARIAL Y DE SISTEMAS,NINGUNA,"Asoc. 8 De Noviembre Mz D Lt 04, distrito de S...",wilberedgarlopez@gmail.com,wilber.lopez@usil.pe,Pendiente,51995070696,51952564231,Pendiente,Joel Felix Sapaylle Poma,48016557,"Ca. Gerónimo De Aliaga 287 Urb. Fundo Oyague, ...",joel.sapaylle@gmail.com,51986948664,No aplica,No aplica,No aplica,No aplica,No aplica,MASCULINO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,"ARIAS RIVERA, NICOLE MADELEYNE",No tiene carpeta,74842555.0,1920282,ACTIVO,NO APLICA,NO APLICA,NO APLICA,NO APLICA,-,No enviado,NaT,No,2019-02,USIL,PREGRADO REGULAR COAR,PSICOLOGÍA,NINGUNA,"Asoc. Perecanacuy Mz G Lt 7, distrito de Vill...",nicolemadeleyne22@gmail.com,nicole.arias@usil.pe,Pendiente,51902595423,Pendiente,Pendiente,Carlos Teodoro Ventura Rivera,8395236,"Asent. H. ""Los Cuatro Suyos"" Mz. B Lt. 10, dis...",rivera.nazar15@gmail.com,51933155579,No aplica,No aplica,No aplica,No aplica,No aplica,FEMENINO
1076,"CALLE ABAD, NAYELI SOLEDAD",No tiene carpeta,75199677.0,1920167,INACTIVO,RETIRO VOLUNTARIO FINALIZAD

# df_consolidado

In [151]:
df_consolidado.head()

,AñoMes,Año,Mes,Apellidos y nombres del Beneficiario,N° DNI,Centro de Estudios,Promoción GEF,Programa académico,Apellido y nombres titular de boleta,DNI del titular de la boleta,N° Boleta,Tipo de cuota emitida,Capital del mes,Interés del mes,Desgravamen del mes,Costo de gestión del mes,Cuota del mes,Monto enviado al banco,Fecha de vencimiento de la cuota,¿Pagó la cuota?,Estado de la cuota,Monto pagado de la cuota\n(soles),Diferencia entre monto facturado y pagado de la cuota (soles),Fundamento de diferencia entre montos,Fecha de pago de la cuota,Canal de pago,Número de días de atraso en el pago de la cuota,"Monto de mora pagada en la cuota (Según Teletransfer, soles)","Monto de penalidad pagada en la cuota (Según Teletransfer, soles)",Observaciones,Mesdni
0,202409,2024,9,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00015069,Estudiante,381.64,118.36,4.59,40.0,544.59,544.59,2024-09-30,SÍ,PAGADA,544.59,0.0,No existió diferencia entre el monto emitido y...,2024-09-28,BCP,-2,0.0,0.0,Ninguna,20240973050262
1,202410,2024,10,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00015926,Estudiante,382.47,117.53,4.41,40.0,544.41,544.41,2024-10-31,SÍ,PAGADA,544.41,0.0,No existió diferencia entre el monto emitido y...,2024-10-25,BCP,-6,0.0,0.0,Ninguna,20241073050262
2,202411,2024,11,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00016768,Estudiante,390.93,109.07,4.23,40.0,544.23,544.23,2024-11-30,SÍ,PAGADA,544.23,0.0,No existió diferencia entre el monto emitido y...,2024-11-25,BCP,-5,0.0,0.0,Ninguna,20241173050262
3,202412,2024,12,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00017606,Estudiante,392.19,107.81,4.05,40.0,544.05,544.05,2024-12-31,SÍ,PAGADA,544.05,0.0,No existió diferencia entre el monto emitido y...,2025-01-01,BCP,1,0.0,0.0,Crep 31 de diciembre,20241273050262
4,202501,2025,1,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00018442,Estudiante,396.84,103.16,3.86,40.0,543.86,543.86,2025-01-31,PENDIENTE,PENDIENTE,NaN,NaN,NaN,NaT,NaN,PENDIENTE,NaN,NaN,NaN,20250173050262


In [152]:
df_consolidado.columns = clean_column_names(df_consolidado.columns)
df_consolidado

,ANOMES,ANO,MES,APELLIDOS_Y_NOMBRES_DEL_BENEFICIARIO,Ndeg_DNI,CENTRO_DE_ESTUDIOS,PROMOCION_GEF,PROGRAMA_ACADEMICO,APELLIDO_Y_NOMBRES_TITULAR_DE_BOLETA,DNI_DEL_TITULAR_DE_LA_BOLETA,Ndeg_BOLETA,TIPO_DE_CUOTA_EMITIDA,CAPITAL_DEL_MES,INTERES_DEL_MES,DESGRAVAMEN_DEL_MES,COSTO_DE_GESTION_DEL_MES,CUOTA_DEL_MES,MONTO_ENVIADO_AL_BANCO,FECHA_DE_VENCIMIENTO_DE_LA_CUOTA,?PAGO_LA_CUOTA?,ESTADO_DE_LA_CUOTA,MONTO_PAGADO_DE_LA_CUOTA\n(SOLES),DIFERENCIA_ENTRE_MONTO_FACTURADO_Y_PAGADO_DE_LA_CUOTA_(SOLES),FUNDAMENTO_DE_DIFERENCIA_ENTRE_MONTOS,FECHA_DE_PAGO_DE_LA_CUOTA,CANAL_DE_PAGO,NUMERO_DE_DIAS_DE_ATRASO_EN_EL_PAGO_DE_LA_CUOTA,"MONTO_DE_MORA_PAGADA_EN_LA_CUOTA_(SEGUN_TELETRANSFER,_SOLES)","MONTO_DE_PENALIDAD_PAGADA_EN_LA_CUOTA_(SEGUN_TELETRANSFER,_SOLES)",OBSERVACIONES,MESDNI
0,202409,2024,9,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00015069,Estudiante,381.64,118.36,4.59,40.0,544.59,544.59,2024-09-30,SÍ,PAGADA,544.59,0.00,No existió diferencia entre el monto emitido y...,2024-09-28,BCP,-2,0.00,0.0,Ninguna,20240973050262
1,202410,2024,10,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00015926,Estudiante,382.47,117.53,4.41,40.0,544.41,544.41,2024-10-31,SÍ,PAGADA,544.41,0.00,No existió diferencia entre el monto emitido y...,2024-10-25,BCP,-6,0.00,0.0,Ninguna,20241073050262
2,202411,2024,11,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00016768,Estudiante,390.93,109.07,4.23,40.0,544.23,544.23,2024-11-30,SÍ,PAGADA,544.23,0.00,No existió diferencia entre el monto emitido y...,2024-11-25,BCP,-5,0.00,0.0,Ninguna,20241173050262
3,202412,2024,12,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00017606,Estudiante,392.19,107.81,4.05,40.0,544.05,544.05,2024-12-31,SÍ,PAGADA,544.05,0.00,No existió diferencia entre el monto emitido y...,2025-01-01,BCP,1,0.00,0.0,Crep 31 de diciembre,20241273050262
4,202501,2025,1,"BOJORQUEZ ANCASI, MARY CIELO",73050262,USIL,2024-01,PREGRADO REGULAR COAR,"BOJORQUEZ ANCASI, MARY CIELO",73050262,B B03-00018442,Estudiante,396.84,103.16,3.86,40.0,543.86,543.86,2025-01-31,PENDIENTE,PENDIENTE,NaN,NaN,NaN,NaT,NaN,PENDIENTE,NaN,NaN,NaN,20250173050262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34004,202409,2024,9,"ZUÑIGA ROBLES, MARIA FERNANDA",73671322,USIL,2023-01,CPEL,"ZUÑIGA ROBLES, MARIA FERNANDA",73671322,B B03-00014799,Estudiante,109.75,240.25,4.19,40.0,394.19,394.19,2024-09-30,SÍ,PAGADA,394.19,0.00,No existió diferencia entre el monto emitido y...,2024-09-30,BCP,0,0.00,0.0,Ninguna,20240973671322
34005,202410,2024,10,"ZUÑIGA ROBLES, MARIA FERNANDA",73671322,USIL,2023-01,CPEL,"ZUÑIGA ROBLES, MARIA FERNANDA",73671322,B B03-00015652,Estudiante,103.07,246.93,4.17,40.0,394.17,394.17,2024-10-31,SÍ,PAGADA,446.34,52.17,Pagó mora y penalidad por deuda atrasada,2024-11-27,BCP,27,4.17,48.0,Ninguna,20241073671322
34006,202411,2024,11,"ZUÑIGA ROBLES, MARIA FERNANDA",73671322,USIL,2023-01,CPEL,"ZUÑIGA ROBLES, MARIA FERNANDA",73671322,B B03-00016504,Estudiante,112.34,237.66,4.15,40.0,394.15,394.15,2024-11-30,SÍ,PAGADA,443.69,49.54,Pagó mora y penalidad por deuda atrasada,2024-12-10,YAPE,10,1.54,48.0,Ninguna,20241173671322
34007,202412,2024,12,"ZUÑIGA ROBLES, MARIA FERNANDA",73671322,USIL,2023-01,CPEL,"ZUÑIGA ROBLES, MARIA FERNANDA",73671322,B B03-00017345,Estudiante,105.78,244.22,4.12,40.0,394.12,394.12,2024-12-31,SÍ,PAGADA,394.12,0.00,No existió diferencia entre el monto emitido y...,2024-12-30,BCP,-1,0.00,0.0,Ninguna,20241273671322


In [153]:
df_consolidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34009 entries, 0 to 34008
Data columns (total 31 columns):
 #   Column                                                             Non-Null Count  Dtype         
---  ------                                                             --------------  -----         
 0   ANOMES                                                             34009 non-null  int64         
 1   ANO                                                                34009 non-null  int64         
 2   MES                                                                34009 non-null  int64         
 3   APELLIDOS_Y_NOMBRES_DEL_BENEFICIARIO                               34009 non-null  object        
 4   Ndeg_DNI                                                           34009 non-null  int64         
 5   CENTRO_DE_ESTUDIOS                                                 34009 non-null  object        
 6   PROMOCION_GEF                                                 

Vamos a quedarnos con años mayores o iguales a 2021, puesto que por motivos de Pandemia en Covid-19 sería recomendable hacer el estudio de forma separada

In [154]:
df_consolidado_filtrado = df_consolidado[df_consolidado["ANO"] > 2021]

En base al diccionario de datos, colocamos las categorías correspondientes para columna

In [157]:
var_categorica = [
    "ANOMES", "ANO", "MES", "APELLIDOS_Y_NOMBRES_DEL_BENEFICIARIO",
    "Ndeg_DNI", "CENTRO_DE_ESTUDIOS", "PROMOCION_GEF", "PROGRAMA_ACADEMICO",
    "APELLIDO_Y_NOMBRES_TITULAR_DE_BOLETA", "TIPO_DE_CUOTA_EMITIDA",
    "?PAGO_LA_CUOTA?", "ESTADO_DE_LA_CUOTA", "FUNDAMENTO_DE_DIFERENCIA_ENTRE_MONTOS",
    "CANAL_DE_PAGO", "OBSERVACIONES", "MESDNI", "NUMERO_DE_DIAS_DE_ATRASO_EN_EL_PAGO_DE_LA_CUOTA",
]

var_numerica = [
    "CAPITAL_DEL_MES", "INTERES_DEL_MES", "DESGRAVAMEN_DEL_MES",
    "COSTO_DE_GESTION_DEL_MES", "CUOTA_DEL_MES", "MONTO_ENVIADO_AL_BANCO",
    "MONTO_DE_MORA_PAGADA_EN_LA_CUOTA_(SEGUN_TELETRANSFER,_SOLES)",
    "MONTO_DE_PENALIDAD_PAGADA_EN_LA_CUOTA_(SEGUN_TELETRANSFER,_SOLES)"
]

var_id = [
    "DNI_DEL_TITULAR_DE_LA_BOLETA", "NDEG_BOLETA"
]

var_fecha = [
    "FECHA_DE_VENCIMIENTO_DE_LA_CUOTA", "FECHA_DE_PAGO_DE_LA_CUOTA"
]


In [158]:
# Convertir variables categóricas a tipo 'object'
df_consolidado[var_categorica] = df_consolidado[var_categorica].astype('object')

# Convertir variables numéricas a tipo 'float64'
df_consolidado[var_numerica] = df_consolidado[var_numerica].astype('float64')

# Convertir IDs a tipo 'category'
df_consolidado[var_id] = df_consolidado[var_id].astype('category')

# Asegurar que las variables de fecha sigan siendo datetime
df_consolidado[var_fecha] = df_consolidado[var_fecha].apply(pd.to_datetime)


KeyError: "['NDEG_BOLETA'] not in index"

In [159]:
import numpy as np

def porcentaje_outlier(df, column):
    rango_intercuartil = df[column].quantile(0.75) - df[column].quantile(0.25)
    limite_superior = df[column].quantile(0.75) + 1.5 * rango_intercuartil
    limite_inferior = df[column].quantile(0.25) - 1.5 * rango_intercuartil

    outliers_count = ((df[column] < limite_inferior) | (df[column] > limite_superior)).sum()
    porcentaje_outliers = np.round(outliers_count * 100 / df.shape[0], 2)

    return (
        f"Columna: {column}\n"
        f"Límite superior: {limite_superior}\n"
        f"Límite inferior: {limite_inferior}\n"
        f"Porcentaje de outliers: {porcentaje_outliers}%\n"
    )

# % de outliers
for columna_num in var_numerica:
    print(porcentaje_outlier(df_consolidado_filtrado, columna_num))


Columna: CAPITAL_DEL_MES
Límite superior: 755.605
Límite inferior: -174.11499999999998
Porcentaje de outliers: 5.18%

Columna: INTERES_DEL_MES
Límite superior: 359.53999999999996
Límite inferior: -100.54
Porcentaje de outliers: 3.72%

Columna: DESGRAVAMEN_DEL_MES
Límite superior: 9.193340714999998
Límite inferior: -2.7480044289999994
Porcentaje de outliers: 4.62%

Columna: COSTO_DE_GESTION_DEL_MES
Límite superior: 40.0
Límite inferior: 40.0
Porcentaje de outliers: 4.75%

Columna: CUOTA_DEL_MES
Límite superior: 938.89
Límite inferior: -16.79000000000002
Porcentaje de outliers: 6.22%

Columna: MONTO_ENVIADO_AL_BANCO
Límite superior: 938.865
Límite inferior: -16.775000000000034
Porcentaje de outliers: 6.21%

Columna: MONTO_DE_MORA_PAGADA_EN_LA_CUOTA_(SEGUN_TELETRANSFER,_SOLES)
Límite superior: 1.7249999999998522
Límite inferior: -1.0349999999999113
Porcentaje de outliers: 14.48%

Columna: MONTO_DE_PENALIDAD_PAGADA_EN_LA_CUOTA_(SEGUN_TELETRANSFER,_SOLES)
Límite superior: 0.0
Límite inferio